In [66]:
import numpy as np
import pandas as pd


def calculate_metric(
    df, value_name, user_id_name, list_user_id, date_name, period, metric_name
):
    """Вычисляет значение метрики для списка пользователей в определённый период.
    
    df - pd.DataFrame, датафрейм с данными
    value_name - str, название столбца со значениями для вычисления целевой метрики
    user_id_name - str, название столбца с идентификаторами пользователей
    list_user_id - List[int], список идентификаторов пользователей, для которых нужно посчитать метрики
    date_name - str, название столбца с датами
    period - dict, словарь с датами начала и конца периода, за который нужно посчитать метрики.
        Пример, {'begin': '2020-01-01', 'end': '2020-01-08'}. Дата начала периода входит нужный
        полуинтервал, а дата окончание нет, то есть '2020-01-01' <= date < '2020-01-08'.
    metric_name - str, название полученной метрики

    return - pd.DataFrame, со столбцами [user_id_name, metric_name], кол-во строк должно быть равно
        кол-ву элементов в списке list_user_id.
    """
    # YOUR_CODE_HERE 
    date_filter = (period['begin'] <= df[date_name]) & (df[date_name] < period['end'])
    sub_df = df[date_filter]
    
    sub_df = sub_df.query(f"{user_id_name} in @list_user_id")
    
    ans = sub_df.groupby(user_id_name)[value_name].sum().to_dict()
    print(ans)
    for k in list_user_id:
        if k not in ans:
            ans[k] = 0

    ans = pd.DataFrame.from_dict(ans, orient='index').reset_index()
    ans.columns = [user_id_name, metric_name]
    return ans



def calculate_metric_cuped(
    df, value_name, user_id_name, list_user_id, date_name, periods, metric_name
):
    """Вычисляет метрики во время пилота, коварианту и преобразованную метрику cuped.
    
    df - pd.DataFrame, датафрейм с данными
    value_name - str, название столбца со значениями для вычисления целевой метрики
    user_id_name - str, название столбца с идентификаторами пользователей
    list_user_id - List[int], список идентификаторов пользователей, для которых нужно посчитать метрики
    date_name - str, название столбца с датами
    periods - dict, словарь с датами начала и конца периода пилота и препилота.
        Пример, {
            'prepilot': {'begin': '2020-01-01', 'end': '2020-01-08'},
            'pilot': {'begin': '2020-01-08', 'end': '2020-01-15'}
        }.
        Дата начала периода входит в полуинтервал, а дата окончания нет,
        то есть '2020-01-01' <= date < '2020-01-08'.
    metric_name - str, название полученной метрики

    return - pd.DataFrame, со столбцами
        [user_id_name, metric_name, f'{metric_name}_prepilot', f'{metric_name}_cuped'],
        кол-во строк должно быть равно кол-ву элементов в списке list_user_id.
    """
    # YOUR_CODE_HERE
    df_prepilot = calculate_metric(df, value_name, user_id_name, list_user_id, date_name, 
                                   periods['prepilot'], f'{metric_name}_prepilot')
    df_pilot = calculate_metric(df, value_name, user_id_name, list_user_id, date_name, 
                                periods['pilot'], metric_name)
    
    df_combo = df_pilot.merge(df_prepilot, on=user_id_name)
    df_combo.fillna(0, inplace=True)
    
    cov = np.cov([df_combo[f'{metric_name}_prepilot'], df_combo[metric_name]])[0, 1]
    theta = cov / np.var(df_combo[f'{metric_name}_prepilot'])
    
    df_combo[f'{metric_name}_cuped'] = df_combo[metric_name] - theta * df_combo[f'{metric_name}_prepilot']
    return df_combo

In [51]:
size = 20
value_name2 = 'value_name2'
user_id_name2 = 'user_id_name2'
date_name2 = 'date_name2'
period2 = {'begin': pd.to_datetime('2017-05-05'), 'end': pd.to_datetime('2019-09-09')}

df = pd.DataFrame({value_name2: np.random.randint(1,20,size=size),
                   user_id_name2: np.random.choice(np.unique(np.random.randint(5000,5010, size**2)), 
                                                    replace=True, size=size),
                   date_name2: np.random.choice([x.date() for x in pd.date_range('2019-01-01', '2020-01-01')], 
                                                 replace=True, size=size)})
df

,value_name2,user_id_name2,date_name2
0,6,5006,2019-11-13
1,1,5006,2019-11-14
2,15,5006,2019-09-18
3,8,5006,2019-01-16
4,18,5006,2019-07-30
5,3,5002,2019-09-26
6,19,5000,2019-10-08
7,3,5004,2019-11-21
8,17,5000,2019-11-05
9,16,5007,2019-04-02


In [52]:
metric_name2 = 'metric_name2'

In [72]:
list_user_id2 = np.random.choice(df[user_id_name2].unique(), replace=False, size=3)
list_user_id2

array([5002, 5006, 5004])

In [54]:
ans = calculate_metric(df, value_name2, user_id_name2, list_user_id2, date_name2, period2, metric_name2)
ans

{5003: 19, 5006: 56, 5007: 31}


,user_id_name2,metric_name2
0,5003,19
1,5006,56
2,5007,31


In [73]:
periods = {'prepilot': period2, 
           'pilot': {'begin': pd.to_datetime('2019-10-10'),
                     'end': pd.to_datetime('2020-10-10')}}
calculate_metric_cuped(df, value_name2, user_id_name2, list_user_id2, date_name2, periods, metric_name2)

{5002: 13, 5006: 56}
{5002: 6, 5004: 11, 5006: 7}
-0.07159487776484284 -41.0


,user_id_name2,metric_name2,metric_name2_prepilot,metric_name2_cuped
0,5002,6,13,6.930733
1,5004,11,0,11.000000
2,5006,7,56,11.009313


In [6]:
qwe

NameError: name 'qwe' is not defined

In [ ]:
for k in ['begin', 'end']:
    period[k] = pd.to_datetime(period[k], format='%Y-%m-%d').date()
period

In [ ]:
date_filter = (period['begin'] <= df[date_name]) & (df[date_name] < period['end'])
sub_df = df[date_filter]
sub_df

In [ ]:
sub_df[sub_df.user_id_name.isin(list_user_id).values]

In [ ]:
sub_df = sub_df.query(f"{user_id_name} in @list_user_id")
sub_df

In [ ]:
ans = sub_df.groupby(user_id_name)[value_name].sum().to_dict()#reset_index()
for k in list_user_id:
    if k not in ans:
        ans[k] = 0
        
ans = pd.DataFrame.from_dict(ans, orient='index').reset_index()
ans.columns = [user_id_name, metric_name]

In [ ]:
ans

In [ ]:

ans

In [ ]:
ans.columns = [user_id_name, metric_name]

In [ ]:
ans